In [1]:
import os
import sys
from bs4 import BeautifulSoup
import pprint



In [2]:
import requests
import pandas as pd
import ast 
import re  # 정규 표현식 모듈 추가
import pandas as pd
import time
import random


In [8]:
# 전역 데이터프레임 초기화
final_data = pd.DataFrame()

def gamefaqs_game_diff(game_key, name):
    url = f"https://gamefaqs.gamespot.com/{name}/stats"
    user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    ]
    headers = {
        "User-Agent": random.choice(user_agents),
        "Accept": "application/json",
        "X-Requested-With": "XMLHttpRequest",
    }
    time.sleep(random.uniform(1, 3))
    response = requests.get(url, headers=headers)
    # print(response)    
    difficulty_raw = ''
        
    # 응답 확인
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        summary = soup.select("script")
        for s in summary:
        # 정규식으로 { label: ... }와 Just Right 포함 부분 매칭
            match = re.search(r'\{ label:.*?Unforgiving.*?\}', s.text)
            if match:
                difficulty_raw = match.group()
    if not difficulty_raw:
        print(f"No data found for {game_key}. Returning default values.")
          
    pattern = r'{ label:"(.*?)", data: (\d+), color:"(.*?)" }'
    matches = re.findall(pattern, difficulty_raw)

    parsed_data = []
    levels = {"Simple":0, "Easy":0, "Just Right":0, "Tough":0, "Unforgiving":0}
    for parsed in parsed_data:
        levels[parsed["label"]] = 1
    # print(levels)
    for key, value in levels.items():
        if value == 0:
            parsed_data.append({"label": key, "data": 0})  
    for label, data, color in matches:
        label_text = label.split(" - ")[0]  # "Simple", "Easy", "Just Right" 추출
        parsed_data.append({"label": label_text, "data": int(data)})
    df = pd.DataFrame(parsed_data)
    difficulty_map = {
        "Simple": 1,    "Easy": 2,    "Just Right": 3,    "Tough": 4,    "Unforgiving": 5
    }
    for label in difficulty_map.keys():
        if label not in [d["label"] for d in parsed_data]:
            parsed_data.append({"label": label, "data": 0})
    
        
    other_df = df
    other_df = other_df.rename(columns={"data": "total"})
    other_df["difficulty_id"] = other_df["label"].map(difficulty_map)
    other_df = other_df.pivot(columns="difficulty_id", values="total").fillna(0)
    other_df["game"] = game_key
    other_df = other_df.rename(columns={5: "diff_5", 4: "diff_4", 3: "diff_3", 2: "diff_2", 1: "diff_1"})
    other_df = other_df[["game", "diff_5", "diff_4", "diff_3", "diff_2", "diff_1"]]
    other_df = other_df.groupby("game", as_index=False).sum()
    other_df["total"] = other_df[["diff_5", "diff_4", "diff_3", "diff_2", "diff_1"]].sum(axis=1)
    other_df["average"] = (
        (other_df["diff_5"] * 5 + other_df["diff_4"] * 4 + other_df["diff_3"] * 3 + other_df["diff_2"] * 2 + other_df["diff_1"] * 1)
        / other_df["total"]
        ).round(2)
    return other_df

gamefaqs_game_list = [
    ("오버워치","ps4/182930-overwatch"),
    ("발로란트","pc/285009-valorant"),
    ("클래시오브클랜","iphone/678942-clash-of-clans"),
    ("클래시로얄","android/188972-clash-royale"),
    ("스타크래프트","pc/25418-starcraft"),
    ("캔디크러쉬사가","android/702058-candy-crush-saga"),
    ("마인크래프트","pc/606524-minecraft"),
    # ("모여봐요동물의숲","switch/248082-animal-crossing-new-horizons"),
    # ("포켓몬스터","switch/359435-pokemon-scarlet"),
    # ("피파23","ps5/373098-fifa-23"),
    # ("월드오브워크래프트", "pc/534914-world-of-warcraft"),
    # ("블레이드앤소울","pc/955631-blade-and-soul"),
    # ("마비노기","pc/926429-mabinogi"),
    # ("리그오브레전드","pc/954437-league-of-legends"),
    # ("아레나오브발러","switch/221219-arena-of-valor"),
    # ("도타2","pc/610657-dota-2"),
    # ("배틀그라운드","pc/206545-playerunknowns-battlegrounds"),
    # ("배틀그라운드",'android/235332-pubg-mobile'),
    # ("메이플스토리","pc/924697-maplestory"),
    # ("던전앤파이터","pc/937883-dungeon-fighter-online"),
    # ("로블록스","pc/185091-roblox"),
    # ("브롤스타즈", "android/254094-brawl-stars"),
    # ("브롤스타즈", "iphone/253930-brawl-stars"),
    # ("NBA2K","ps5/394854-mlb-the-show-23"),
    # ("쿠키런킹덤",'android/332328-cookie-run-kingdom'),
    # ("쿠키런킹덤(i)","iphone/330536-cookie-run-kingdom"),
    # ("서든어택","pc/960351-sudden-attack"),
    # ("테일즈러너","pc/952467-tales-runner"),
    # ("카트라이더","pc/939366-kartrider")
]
# print(final_data)
for game_key, name in gamefaqs_game_list:
    gamefaqs_game_data = gamefaqs_game_diff(game_key, name)
    final_data = pd.concat([final_data, gamefaqs_game_data], ignore_index=True)

# 최종 데이터 출력
final_data_sorted = final_data.sort_values(by="average", ascending=True)
final_data_sorted


No data found for 쿠키런킹덤. Try request https://gamefaqs.gamespot.com/iphone/330536-cookie-run-kingdom/stats
No data found for 쿠키런킹덤. Try request https://gamefaqs.gamespot.com/iphone/330536-cookie-run-kingdom/stats
No data found for 쿠키런킹덤. Try request https://gamefaqs.gamespot.com/iphone/330536-cookie-run-kingdom/stats
No data found for 쿠키런킹덤. Try request https://gamefaqs.gamespot.com/iphone/330536-cookie-run-kingdom/stats
No data found for 쿠키런킹덤. Try request https://gamefaqs.gamespot.com/iphone/330536-cookie-run-kingdom/stats
No data found for 쿠키런킹덤. Try request https://gamefaqs.gamespot.com/iphone/330536-cookie-run-kingdom/stats
No data found for 쿠키런킹덤. Try request https://gamefaqs.gamespot.com/iphone/330536-cookie-run-kingdom/stats
No data found for 쿠키런킹덤. Try request https://gamefaqs.gamespot.com/iphone/330536-cookie-run-kingdom/stats
No data found for 쿠키런킹덤. Try request https://gamefaqs.gamespot.com/iphone/330536-cookie-run-kingdom/stats
No data found for 쿠키런킹덤. Try request https://g

KeyboardInterrupt: 

In [5]:
# final_data_time = pd.DataFrame()

def gamefaqs_game_time(game_key, name):
    url = f"https://gamefaqs.gamespot.com/{name}/stats"
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    ]
    headers = {
        "User-Agent": random.choice(user_agents),
        "Accept": "application/json",
        "X-Requested-With": "XMLHttpRequest",
    }
    response = requests.get(url, headers=headers)
    time.sleep(random.uniform(1, 3))
    response = requests.get(url, headers=headers)
    # print(response)    
    play_time_raw = ''
    # 응답 확인
    summary = []
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        summary = soup.select("script") 
    for idx, s in enumerate(summary):  # idx는 인덱스, s는 요소
        match = re.search(r'\{ label:.*?80 Hours.*?\}', s.text)  # 이제 s.text가 올바르게 작동
        if match:
            play_time_raw = match.group() 
    ###############################################################잘 나옴#######################################
    if not play_time_raw:
        print(f"{game_key}의 데이터가 없음")
    pattern = r'{ label:"(.*?)", data: (\d+), color:"(.*?)" }'
    matches = re.findall(pattern, play_time_raw)

    parsed_data2 = []
    times = {"&lt; 1 Hour":0, "~1 Hour":0, "~2 Hours":0, "~4 Hours":0, "~8 Hours":0,"~12 Hours":0,"~20 Hours":0,"~40 Hours":0,"~60 Hours":0, "&gt;= 80 Hours":0}
    for parsed in parsed_data2:
        times[parsed["label"]] = 1
    # print(times)
    for key, value in times.items():
        if value == 0:
            parsed_data2.append({"label": key, "data": 0})  
    # print(parsed_data2)
    for label, data, color in matches:
        label_text = label.split(" - ")[0]  # "Simple", "Easy", "Just Right" 추출
        parsed_data2.append({"label": label_text, "data": int(data)})
    play_time_df = pd.DataFrame(parsed_data2)
    play_time_df
    play_time_map = {
        "&lt; 1 Hour": 1,    "~1 Hour": 2,    "~2 Hours": 3,    "~4 Hours": 4,    "~8 Hours": 5,   "~12 Hours": 6,    "~20 Hours": 7,    "~40 Hours": 8,    "~60 Hours": 9,    "&gt;= 80 Hours": 10
    }
    for label in play_time_map.keys():
        if label not in [d["label"] for d in parsed_data2]:
            parsed_data2.append({"label": label, "data": 0})

    play_time_df = play_time_df.rename(columns={"data": "total"})
    play_time_df["play_time_idx"] = play_time_df["label"].map(play_time_map)
    play_time_df = play_time_df.pivot(columns="play_time_idx", values="total").fillna(0)
    play_time_df["game"] = game_key
    play_time_df = play_time_df.rename(columns={1: "< 1 Hour", 2: "~1 Hour", 3: "~2 Hours", 4: "~4 Hours", 5: "~8 Hours", 6: "~12 Hours", 7: "~20 Hours", 8: "~40 Hours", 9: "~60 Hours", 10: "80 Hours"})
    play_time_df = play_time_df[["game", "< 1 Hour", "~1 Hour", "~2 Hours", "~4 Hours", "~8 Hours", "~12 Hours", "~20 Hours", "~40 Hours", "~60 Hours", "80 Hours"]]
    play_time_df = play_time_df.groupby("game", as_index=False).sum()
    play_time_df["time_total"] = play_time_df[["< 1 Hour", "~1 Hour", "~2 Hours", "~4 Hours", "~8 Hours", "~12 Hours", "~20 Hours", "~40 Hours", "~60 Hours", "80 Hours"]].sum(axis=1)
    play_time_df["time_average"] = (
    (play_time_df["< 1 Hour"] * 0 + play_time_df["~1 Hour"] * 1 + play_time_df["~2 Hours"] * 2 + play_time_df["~4 Hours"] * 3 + play_time_df["~8 Hours"] * 4 + play_time_df["~12 Hours"] * 5 + play_time_df["~20 Hours"] * 6 + play_time_df["~40 Hours"] * 7 + play_time_df["~60 Hours"] * 8 + play_time_df["80 Hours"] * 9)
    / play_time_df["time_total"]
    ).round(2)
    return play_time_df

for game_key, name in gamefaqs_game_list:
    gamefaqs_time_data = gamefaqs_game_time(game_key, name)
    print(gamefaqs_time_data)
#     final_data_time = pd.merge(final_data, gamefaqs_time_data, on="game", how="left")
# final_data_time
# final_data_sorted
# 최종 데이터 출력
# final_data_sorted = final_data.sort_values(by="time_average", ascending=True)
# final_data_sorted

play_time_idx  game  < 1 Hour  ~1 Hour  ~2 Hours  ~4 Hours  ~8 Hours  \
0              오버워치      14.0      2.0       2.0       5.0       8.0   

play_time_idx  ~12 Hours  ~20 Hours  ~40 Hours  ~60 Hours  80 Hours  \
0                   17.0       23.0       48.0       42.0     400.0   

play_time_idx  time_total  time_average  
0                   561.0          8.11  
play_time_idx  game  < 1 Hour  ~1 Hour  ~2 Hours  ~4 Hours  ~8 Hours  \
0              발로란트       1.0      1.0       2.0       3.0       0.0   

play_time_idx  ~12 Hours  ~20 Hours  ~40 Hours  ~60 Hours  80 Hours  \
0                    5.0        4.0        2.0        8.0      50.0   

play_time_idx  time_total  time_average  
0                    76.0          7.78  
play_time_idx     game  < 1 Hour  ~1 Hour  ~2 Hours  ~4 Hours  ~8 Hours  \
0              클래시오브클랜       0.0      1.0       2.0       1.0       1.0   

play_time_idx  ~12 Hours  ~20 Hours  ~40 Hours  ~60 Hours  80 Hours  \
0                    0.0        2.

In [ ]:
for idx, s in enumerate(summary):
    match = re.search(r'\{ label:.*?80 Hours.*?\}', s.text)
    if match:  # 매칭된 데이터가 있을 경우만 출력
        print(f"Matched data:\n{match.group()}\n")